<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_AutoEncoder-with-LSTM/blob/main/Ensemble_Bidirectional_LSTM_Denoising_AE_230426.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun,JANG
# limitsinx.tistory.com
# Last revision date : 2023.04.26

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

xy_train = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/train_v.txt',delimiter=',',dtype=np.float32) #5808
xy_test = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/test_v.txt',delimiter=',',dtype=np.float32) #10883
xy_train = np.delete(xy_train ,3, axis = 1)
xy_test = np.delete(xy_test ,3, axis = 1)

start_test = 1000
end_test = 5000
xy_test_1 = xy_test[:start_test,:]
xy_test_1n2 = xy_test[start_test:end_test,:]
xy_test_2 = xy_test[end_test:,:]

xy_train = np.concatenate((xy_train, xy_test_1n2), axis = 0)
xy_test = np.concatenate((xy_test_1, xy_test_2), axis = 0)

def MinMaxScaler(data) : # Normal Std
  numerator = data - np.min(data,0)
  denominator = np.max(data,0) - np.min(data,0)

  return numerator / (denominator + 1e-7)

def origin_minmax(data) : # HJ STYLE
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])

  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max_train, arr_min_train = origin_minmax(xy_train)
arr_max_test, arr_min_test = origin_minmax(xy_test)
arr_max_total = arr_max_train
arr_min_total = arr_min_train
print(arr_max_train, arr_min_train)
print(arr_max_test, arr_min_test)

for i in range(xy_train.shape[1]) : 
  print(i)
  if(arr_max_train[i] > arr_max_test[i]) : 
    arr_max_total[i] = arr_max_train[i]
  else : 
    arr_max_total[i] = arr_max_test[i]
  pass

for i in range(xy_train.shape[1]) : 
  if(arr_min_train[i] < arr_min_test[i]) : 
    arr_min_total[i] = arr_min_train[i]
  else : 
    arr_min_total[i] = arr_min_test[i]
  pass

print(arr_max_total, arr_min_total)

def HJ_MinMaxScaler(data) : 
  for i in range(data.shape[1]) : 
    numerator = data[:,i] - arr_min_total[i]
    denominator = arr_max_total[i] - arr_min_total[i]
    data[:,i] = numerator / (denominator + 1e-7)
    
  return data

def HJ_backMinMax(data) : 
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (arr_max_total[i] - arr_min_total[i]) + arr_min_total[i]

  return data

xy_train_scale = HJ_MinMaxScaler(xy_train)
xy_test_scale = HJ_MinMaxScaler(xy_test)


In [ ]:
import numpy as np

def add_awgn_noise(signal, snr):
    signal = np.array(signal)
    signal_power = np.mean(np.square(signal))
    noise_power = signal_power / (10 ** (snr / 10))
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    return signal + noise

#signal_value = np.ones(4)
signal_value = [-13.700012,  25,       100,       746.3] 
snr_values = [5, 10, 15, 20, 25, 30]

print("Original Data : ", signal_value)
for snr in snr_values:
    noisy_value = add_awgn_noise(signal_value, snr)
    print(f"SNR {snr} dB에 대한 값: {noisy_value}")

In [ ]:
def simulate_awgn(signal, snr, num_trials=1000):
    noisy_signals = []
    for _ in range(num_trials):
        noisy_signal = add_awgn_noise(signal, snr)
        noisy_signals.append(noisy_signal)
    return np.array(noisy_signals)

original_data = np.array([0.5, 0.5, 0.5, 0.5, 0.5])
snr_values = [5, 10, 15, 20, 25, 30]

print("Original Data : ", original_data)
for snr in snr_values:
    simulated_data = simulate_awgn(original_data, snr)
    mean_data = np.mean(simulated_data, axis=0)
    std_data = np.std(simulated_data, axis=0)
    print(f"SNR {snr} dB에 대한 평균 값: {mean_data}")
    print(f"SNR {snr} dB에 대한 표준 편차: {std_data}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# AWGN

def add_awgn_noise(signal, snr):
    signal_power = np.mean(np.square(signal))
    noise_power = signal_power / (10 ** (snr / 10))
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    return signal + noise

SNR_DB = 30

class AWGNCallback(Callback) : 
  def __init__(self, snr_db) : 
    self.snr_db = snr_db
    
  def on_epoch_begin(self, epoch, logs = None) : 
    global trainX
    trainX_noisy = add_awgn_noise(trainX, self.snr_db)
    trainX = trainX_noisy

# Data Sequence Setting
TIME_STEPS = 5
seq_length = TIME_STEPS

train_set = np.copy(xy_train_scale)
test_set = np.copy(xy_test_scale)

#Build Dataset with Data Augmentation(Zero Aug)
def build_dataset(time_series, seq_length) : 
  dataX = []
  dataY = []
  augmented_data = []
  num_samples = len(time_series) - seq_length + 1
  for i in range(0, len(time_series) -seq_length) : 
    x = time_series[i:i+seq_length, :]
    y = time_series[i:i+seq_length, :]

    dataX.append(x)
    dataY.append(y)

  return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

print(trainX.shape, trainY.shape) #(# of datas, Time_steps, features)

INPUT_DIM = trainX.shape[2]

In [ ]:
# Designing the Ensemble Model - First Version
'''
LSTM_UNITS = 32

def Bidirectional_LSTM_AE() : 
  # Encoding
  inputs = tf.keras.Input(shape = (TIME_STEPS, INPUT_DIM, ))
  encoder = tf.keras.layers.Bidirectional(LSTM(LSTM_UNITS, return_sequences = True))(inputs) #(, Time_STEPS, UNITS*2)
  encoder = tf.keras.layers.Bidirectional(LSTM(int(LSTM_UNITS/4), return_sequences = True))(encoder) #(, UNITS*2)
  encoder = tf.keras.layers.Bidirectional(LSTM(int(LSTM_UNITS/8), return_sequences = True))(encoder) #(, UNITS*2)
  ensemble_outputs = []
  for i in range(4):
    # Decoding
    decoder = tf.keras.layers.Bidirectional(LSTM(int(LSTM_UNITS/8), return_sequences=True))(encoder)
    decoder = tf.keras.layers.Bidirectional(LSTM(int(LSTM_UNITS/4), return_sequences=True))(decoder)
    decoder = tf.keras.layers.Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(decoder)
    output = tf.keras.layers.Dense(INPUT_DIM, activation='sigmoid')(decoder)
    ensemble_outputs.append(output)

  # Combine ensemble outputs using average
  ensemble_output = tf.keras.layers.Average()(ensemble_outputs)

  model = tf.keras.Model(inputs=[inputs], outputs=ensemble_output)
  #decoder = tf.keras.layers.Bidirectional(LSTM(int(LSTM_UNITS/8), return_sequences = True))(encoder)
  #decoder = tf.keras.layers.Bidirectional(LSTM(int(LSTM_UNITS/4), return_sequences = True))(decoder)
  #decoder = tf.keras.layers.Bidirectional(LSTM(LSTM_UNITS, return_sequences = True))(decoder)
  #output = tf.keras.layers.Dense(INPUT_DIM, activation = 'sigmoid')(decoder)
  #model = tf.keras.Model(inputs = [inputs], outputs = output)

  return model
  '''

In [ ]:
# Designing the Ensemble Model - Second Version

import tensorflow as tf
from tensorflow.keras.layers import LSTM

LSTM_UNITS = 32
Decoder_count = 2

def Bidirectional_LSTM_AE():
    inputs = tf.keras.Input(shape=(TIME_STEPS, INPUT_DIM,))
    encoder = tf.keras.layers.Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(inputs)
    encoder = tf.keras.layers.Bidirectional(LSTM(int(LSTM_UNITS/4), return_sequences=True))(encoder)
    encoder = tf.keras.layers.Bidirectional(LSTM(int(LSTM_UNITS/4), return_sequences=True))(encoder)

    return encoder, inputs

def create_ensemble(encoder, inputs):
    ensemble_outputs = []
    
    for i in range(Decoder_count):
        decoder = tf.keras.layers.Bidirectional(LSTM(int(LSTM_UNITS/4), return_sequences=True))(encoder)
        decoder = tf.keras.layers.Bidirectional(LSTM(int(LSTM_UNITS/4), return_sequences=True))(decoder)
        decoder = tf.keras.layers.Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(decoder)
        output = tf.keras.layers.Dense(INPUT_DIM, activation='sigmoid')(decoder)
        model = tf.keras.Model(inputs=inputs, outputs=output)
        ensemble_outputs.append(model)
        
    return ensemble_outputs

encoder, inputs = Bidirectional_LSTM_AE()
ensemble_models = create_ensemble(encoder, inputs)


In [ ]:
# Run the First Model
'''
EarlyStopping(monitor = 'loss', min_delta = 0, patience = 3,  mode = 'auto')
early_stopping = EarlyStopping()
awgn_callback = AWGNCallback(SNR_DB)

modelhj = Bidirectional_LSTM_AE()
modelhj.compile(optimizer = 'adam', loss = 'mse')
modelhj.fit(trainX, trainY, epochs = 20, batch_size = 32, callbacks = [early_stopping, awgn_callback])
'''

In [ ]:
# Run the Second Model
EarlyStopping(monitor = 'loss', min_delta = 0, patience = 3,  mode = 'auto')
early_stopping = EarlyStopping()
awgn_callback = AWGNCallback(SNR_DB)

# First Encoder-Decoder Training
first_decoder = ensemble_models[0]
first_decoder.compile(optimizer='adam', loss='mse')
first_decoder.fit(trainX, trainY, epochs=30, batch_size = 16, callbacks=[early_stopping])
#first_decoder.fit(trainX, trainY, epochs=5, batch_size=32, callbacks=[early_stopping, awgn_callback])

# Encoder Freezing after first AE training
encoder.trainable = False
print("Encoder Training is over")

# Residual Decoder Training
for i in range(1, Decoder_count):
    model_i = ensemble_models[i]
    model_i.compile(optimizer='adam', loss='mse')
    model_i.fit(trainX, trainY, epochs=30, batch_size = 32, callbacks=[early_stopping])
    #model_i.fit(trainX, trainY, epochs=10, batch_size=32, callbacks=[early_stopping, awgn_callback])
    print("Decoder ", i+1, "th Training is over")

In [ ]:
def ensemble_predict(testX, ensemble_models):
    predictions = []

    for model in ensemble_models:
        model_prediction = model.predict(testX)
        predictions.append(model_prediction)
        print("model_prediction : ",model_prediction.shape, model_prediction)
        print("prediction size : ",np.array(predictions).shape)
    ensemble_prediction = np.mean(predictions, axis=0)
    return ensemble_prediction

final_output = ensemble_predict(trainX, ensemble_models)

print("Ensemble prediction Average:", final_output.shape, final_output)
print("TrainX : ",trainX.shape, trainX)

In [ ]:
# Training
trainY_predict = np.copy(final_output)
print(trainY_predict.shape)
trainY_predict_2 = trainY_predict.reshape(-1, trainY_predict.shape[-1])
print(trainY_predict_2.shape)
trainY_predict_back = HJ_backMinMax(trainY_predict_2)
trainY_back_1 = trainY.reshape(-1, trainY.shape[-1])
trainY_back = HJ_backMinMax(trainY_back_1)


In [ ]:
#Plotting Training data

plt.figure(figsize = (12,10))
plt.subplot(2,2,1)
plt.plot(trainY_back[seq_length:,0], color = 'blue')
plt.plot(trainY_predict_back[:,0], color = 'red')
plt.legend(['Real Current','Predicted Current'])

plt.subplot(2,2,2)
plt.plot(trainY_back[seq_length:,1], color = 'blue')
plt.plot(trainY_predict_back[:,1], color = 'red')
plt.legend(['Real Temperature','Predicted Temperature'])

plt.subplot(2,2,3)
plt.plot(trainY_back[seq_length:,2], color = 'blue')
plt.plot(trainY_predict_back[:,2], color = 'red')
plt.legend(['Real SOC','Predicted SOC'])

plt.subplot(2,2,4)
plt.plot(trainY_back[seq_length:,3], color = 'blue')
plt.plot(trainY_predict_back[:,3], color = 'red')
plt.legend(['Real Voltage','Predicted Voltage'])


error_current_1 = trainY_back[:,0] - trainY_predict_back[:,0]
error_temp_1 = trainY_back[:,1] - trainY_predict_back[:,1]
error_soc_1 = trainY_back[:,2] - trainY_predict_back[:,2]
error_voltage_1 = trainY_back[:,3] - trainY_predict_back[:,3]

plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(error_current_1, color = 'green')
plt.legend(['Current Error'])

plt.subplot(2,2,2)
plt.plot(error_temp_1, color = 'green')
plt.legend(['Temperature Error'])
plt.ylim(-2, 2)

plt.subplot(2,2,3)
plt.plot(error_soc_1, color = 'green')
plt.legend(['SOC Error'])
plt.ylim(-10, 10)

plt.subplot(2,2,4)
plt.plot(error_voltage_1, color = 'green')
plt.legend(['Voltage Error'])
plt.ylim(-30, 30)


In [ ]:
#testX = add_awgn_noise(testX, SNR_DB)
test_final_output = ensemble_predict(testX, ensemble_models)
print("Ensemble prediction:", test_final_output)

In [ ]:
# Testing
testY_predict = test_final_output
testY_predict_2 = testY_predict.reshape(-1, testY_predict.shape[-1])
testY_predict_back = HJ_backMinMax(testY_predict_2)
testY_back_1 = testY.reshape(-1, testY.shape[-1])
testY_back = HJ_backMinMax(testY_back_1)

print(testY_predict_back.shape)
print(testY_back.shape)

In [ ]:
#Plotting Test Data
import matplotlib.pyplot as plt

plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(testY_back[seq_length:,0], color = 'blue')
plt.plot(testY_predict_back[:,0], color = 'red')
plt.legend(['Real Current','Predicted Current'])

plt.subplot(2,2,2)
plt.plot(testY_back[seq_length:,1], color = 'blue')
plt.plot(testY_predict_back[:,1], color = 'red')
plt.legend(['Real Temperature','Predicted Temperature'])

plt.subplot(2,2,3)
plt.plot(testY_back[seq_length:,2], color = 'blue')
plt.plot(testY_predict_back[:,2], color = 'red')
plt.legend(['Real SOC','Predicted SOC'])

plt.subplot(2,2,4)
plt.plot(testY_back[seq_length:,3], color = 'blue')
plt.plot(testY_predict_back[:,3], color = 'red')
plt.legend(['Real Voltage','Predicted Voltage'])


error_current_1 = testY_back[:,0] - testY_predict_back[:,0]
error_temp_1 = testY_back[:,1] - testY_predict_back[:,1]
error_soc_1 = testY_back[:,2] - testY_predict_back[:,2]
error_voltage_1 = testY_back[:,3] - testY_predict_back[:,3]

plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(error_current_1, color = 'green')
plt.legend(['Current Error'])

plt.subplot(2,2,2)
plt.plot(error_temp_1, color = 'green')
plt.legend(['Temperature Error'])
plt.ylim(-2, 2)

plt.subplot(2,2,3)
plt.plot(error_soc_1, color = 'green')
plt.legend(['SOC Error'])
plt.ylim(-10, 10)

plt.subplot(2,2,4)
plt.plot(error_voltage_1, color = 'green')
plt.legend(['Voltage Error'])
plt.ylim(-30, 30)



In [ ]:
# First Decoder Verifying
first_model = ensemble_models[0]

# 예측하기
predictions = first_model.predict(testX)

print("Preidction with First Decoder : ", predictions.shape, predictions[0:1,:,:])
print("Test X : ", testX.shape, testX[0:1,:,:])